## Исследование надёжности заёмщиков

Задача от кредитного отдела банка. Проанализировать данные и узнать влияет ли семейное положение и количество детей на погашение кредита в срок. Результаты исследования будут использованы при построении модели "кредитного скоринга", которая будет оценивать способность потенциального клиента вернуть кредит банку.

## Изучение датасета

In [7]:
import pandas as pd
import numpy as np

In [8]:
data = pd.read_csv('data_project_1.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

в двух колонках присутствуют пропуски. 

## Предобработка данных

### Обработка пропусков

In [10]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [19]:
df_isnull = data.query('days_employed == "NaN" & total_income == "NaN"')
df_isnull.shape[0]

2174

Пропуски в двух колонках симметричны

Надо заполнить пропуски в колонках days_employed и total_income.


В days_employed присутствуют отрицательные значения, это странно(стаж не может быть отрицательным).
Пропуски в этом столбике можно заполнить средним или медианным значением
Перед этим переведу отрицательные значения в положительные.

In [6]:
data.loc[data['days_employed'] < 0,'days_employed'] = data['days_employed'] * (-1)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Попробую предположить почему появились отрицательные значения в колонке days_employed. У меня 2 версии.
1. В анкете используется некорректная формулировка, которая вызывает ошибки у людей при заполнении.
2. Возможно в поле со стажем, куда люди вводят информацию есть формула вычитания. Таким образом, если человек вводит интервал времени с какого года по какой он работал то получается, что из меньшего вычитается большее, отсюда такое количество отрицательных значений. Формула там точно используется, хотя бы для перевода годов в дни, люди же не мыслят в этом вопросе днями.

Пример:
Начало работы: 2003 год
Окончание работы: 2021 год (по настоящее время)

Получаем интервал, 2003 - 2021 = -18 лет стаж, переводим в дни 18 * 365 = -6570 дней (без учета високосных годов)

In [21]:
de_median = data['days_employed'].median()
data['days_employed'] = data['days_employed'].fillna(de_median)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Пропуски в days_employed заполнены

Все тоже самое повторяю для колонки total_income

In [22]:
ti_median = data['total_income'].median()
data['total_income'] = data['total_income'].fillna(ti_median)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Замена типа данных

In [24]:
data['days_employed'] = data['days_employed'] / 365
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,0,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,0,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,0,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,2,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


### Обработка дубликатов

In [11]:
data.duplicated().sum()

54

In [12]:
data['purpose'] = data['purpose'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['education'] = data['education'].str.lower()

data.duplicated().sum()

71

In [13]:
#71 дубликат это около 3% данных, можно их удалить 

data = data.drop_duplicates()
data.duplicated().sum()

0

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Вывод
Очистил таблицу от дубликатов и образовавшихся пропусков. Возможной причиной появления дубликатов могут быть, как человеческая ошибка (2 раза ввел данные, потому что не понял отправились они или нет, возможно стоит посмотреть на обратную реакцию формы после заполнения), так и техническая (какие-то проблемы с интернетом или неверный алгоритм)

### Лемматизация

In [15]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

In [16]:
def lemmas_purpose(row):
    lemmas = m.lemmatize(row)
    return lemmas

data['purpose'] = data['purpose'].apply(lemmas_purpose)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,"[покупка, , жилье, \n]"
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,"[приобретение, , автомобиль, \n]"
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,"[покупка, , жилье, \n]"
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,"[дополнительный, , образование, \n]"
4,0,932,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,"[сыграть, , свадьба, \n]"
5,0,2,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,"[покупка, , жилье, \n]"
6,0,7,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,"[операция, , с, , жилье, \n]"
7,0,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,"[образование, \n]"
8,2,18,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,"[на, , проведение, , свадьба, \n]"
9,0,5,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,"[покупка, , жилье, , для, , семья, \n]"


In [17]:
#частоту лемм колонки "purpose"

Counter(data['purpose'].sum())

Counter({'покупка': 5897,
         ' ': 33570,
         'жилье': 4460,
         '\n': 21454,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 478,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'подержанный': 486,
         'получение': 1314,
         'высокий': 1374,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607})

### Вывод
Выполнив лемматизацию можно поделить данные на категории и создать новую колонку. Благодаря тому, что все слова в начальной форме можно будет написать функцию, которая будет присваивать цели категорию в новый столбик. Я выделил следующие категории: Недвижимость, Автомобиль, Образование, Свадьба, Строительство, Ремонт.

### Категоризация данных

In [18]:
def category(row):
    
    for i in row:
    
        if 'недвижимость' in i or 'жилье' in i:
            return 'недвижимость'
    
        if 'автомобиль' in i or 'машина' in i:
            return 'автомобиль'
    
        if 'образование' in i:
            return 'образование'
    
        if 'свадьба' in i:
            return 'свадьба'
    
        if 'строительство' in i:
            return 'строительство'
    
        if 'ремонт' in i:
            return 'ремонт'
        
    return 'потребительский'
    
        
data['category'] = data['purpose'].apply(category)

data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,"[покупка, , жилье, \n]",недвижимость
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,"[приобретение, , автомобиль, \n]",автомобиль
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,"[покупка, , жилье, \n]",недвижимость
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,"[дополнительный, , образование, \n]",образование
4,0,932,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,"[сыграть, , свадьба, \n]",свадьба
5,0,2,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,"[покупка, , жилье, \n]",недвижимость
6,0,7,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,"[операция, , с, , жилье, \n]",недвижимость
7,0,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,"[образование, \n]",образование
8,2,18,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,"[на, , проведение, , свадьба, \n]",свадьба
9,0,5,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,"[покупка, , жилье, , для, , семья, \n]",недвижимость


In [19]:
#посмотрю на что чаще всего берут кредит

data['category'].value_counts()

недвижимость     8326
автомобиль       4306
образование      4013
свадьба          2324
строительство    1878
ремонт            607
Name: category, dtype: int64

## Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [20]:
#всего людей имевших долг по кредиту 8%

data['debt'].value_counts()

0    19713
1     1741
Name: debt, dtype: int64

In [28]:
part_debt1 = data.loc[data['debt'] == 1, 'debt'].sum() / len(data)

print('Доля людей, имевших задолженность по кредиту: {:.1%}'.format(part_debt1))


Доля людей, имевших задолженность по кредиту: 8.1%


In [29]:
no_children = data[(data['children'] == 0) & (data['debt'] == 1)]['children'].count() / data[data['children'] == 0]['children'].count()
have_children = data[(data['children'] > 0) & (data['debt'] == 1)]['children'].count() / data[data['children'] > 0]['children'].count()

print()
print('Доля людей без детей, которые имели задолженности(доля от всех у кого нет детей): {:.1%}'.format(no_children))
print('Доля людей с детьми, которые имели задолженности(доля от всех у кого есть дети): {:.1%}'.format(no_children))


Доля людей без детей, которые имели задолженности(доля от всех у кого нет детей): 7.5%
Доля людей с детьми, которые имели задолженности(доля от всех у кого есть дети): 7.5%


### Вывод

Близкий друг к другу или одинаковый процент у разных гупп, говорит о том, что зависимости между ними нет. Здесь я делаю вывод, что между наличием детей и возвратом кредита в срок зависимости нет.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
#посмотрю, какое количество и какие категории у нас есть
data['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [24]:
fs_marriage = data[(data['family_status'] == 'женат / замужем') & (data['debt'] == 1)]['family_status'].count() / data[data['family_status'] == 'женат / замужем']['family_status'].count()
fs_civil = data[(data['family_status'] == 'гражданский брак') & (data['debt'] == 1)]['family_status'].count() / data[data['family_status'] == 'гражданский брак']['family_status'].count()
fs_nomarriage = data[(data['family_status'] == 'не женат / не замужем') & (data['debt'] == 1)]['family_status'].count() / data[data['family_status'] == 'не женат / не замужем']['family_status'].count()
fs_divorce = data[(data['family_status'] == 'в разводе') & (data['debt'] == 1)]['family_status'].count() / data[data['family_status'] == 'в разводе']['family_status'].count()
fs_widower = data[(data['family_status'] == 'вдовец / вдова') & (data['debt'] == 1)]['family_status'].count() / data[data['family_status'] == 'вдовец / вдова']['family_status'].count()

print()
print('Доля людей в браке, имевших задолженность по кредиту: {:.1%}'.format(fs_marriage))
print('Доля людей в гражданском браке, имевших задолженность по кредиту: {:.1%}'.format(fs_civil))
print('Доля людей не в браке, имевших задолженность по кредиту: {:.1%}'.format(fs_nomarriage))
print('Доля разведенных людей, имевших задолженность по кредиту: {:.1%}'.format(fs_divorce))
print('Доля овдовевших людей, имевших задолженность по кредиту: {:.1%}'.format(fs_widower))
print()


Доля людей в браке, имевших задолженность по кредиту: 7.5%
Доля людей в гражданском браке, имевших задолженность по кредиту: 9.3%
Доля людей не в браке, имевших задолженность по кредиту: 9.8%
Доля разведенных людей, имевших задолженность по кредиту: 7.1%
Доля овдовевших людей, имевших задолженность по кредиту: 6.6%



### Вывод

Если взять за норму среднюю задолженность 8%, то тут есть небольшие отклонения в +/- почти 2%, что может говорить о наличии зависимости между семейным статусом и возвратом кредита в срок.

Например, самая большая доля должников среди людей в гражданском или официальном браке, это можно объяснить тем, что такие люди менее стабильны, на них влияет бОльшее количество факторов, такие как - соры, увольнения(если человек работал вместе с партнером), разводы, раздел имущества или потребность обеспечивать бОльшее количество людей(дети, супруг/супруга). Все это может ухудшать финансовое состояние. В то же время разведенные или овдовевшие люди имеют меньше рисков влияющих на финансовое положение, поэтому доля среди таких групп меньше всего.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [25]:
min = data['total_income'].min()
max = data['total_income'].max()

min, max

(20667, 2265604)

In [30]:
#Для исследования зависимости уровня дохода и возврата кредита в срок, разобью total_income на категории
#Категории определю по официальным данным 2018 года. Очень низкий (18000 - 36000),
#Низкий (36000 - 72000), Средний (72000 - 100000), Выше среднего (100000 - 150000), Высокий (150000 - 250000),
#Очень высокий (250000 - без потолка)

def category_ti(i):
    
    if 36000 >= i > 18000:
        return 'очень низкий'
        
    if 72000 >= i > 36000:
        return 'низкий'
        
    if 100000 >= i > 72000:
        return 'средний'
        
    if 150000 >= i > 100000:
        return 'выше среднего'
        
    if 250000 >= i > 150000:
        return 'высокий'
        
    if 250000 < i:
        return 'очень высокий'
        
        

data['categorys_ti'] = data['total_income'].apply(category_ti)

data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category,categorys_ti
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,"[покупка, , жилье, \n]",недвижимость,очень высокий
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,"[приобретение, , автомобиль, \n]",автомобиль,выше среднего
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,"[покупка, , жилье, \n]",недвижимость,выше среднего
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,"[дополнительный, , образование, \n]",образование,очень высокий
4,0,932,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,"[сыграть, , свадьба, \n]",свадьба,высокий
5,0,2,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,"[покупка, , жилье, \n]",недвижимость,очень высокий
6,0,7,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,"[операция, , с, , жилье, \n]",недвижимость,высокий
7,0,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,"[образование, \n]",образование,выше среднего
8,2,18,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,"[на, , проведение, , свадьба, \n]",свадьба,средний
9,0,5,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,"[покупка, , жилье, , для, , семья, \n]",недвижимость,выше среднего


In [27]:
part_ti_verylow = data[(data['categorys_ti'] == 'очень низкий') & (data['debt'] == 1)]['categorys_ti'].count() / data[data['categorys_ti'] == 'очень низкий']['categorys_ti'].count()
part_ti_low = data[(data['categorys_ti'] == 'низкий') & (data['debt'] == 1)]['categorys_ti'].count() / data[data['categorys_ti'] == 'низкий']['categorys_ti'].count()
part_ti_mean = data[(data['categorys_ti'] == 'средний') & (data['debt'] == 1)]['categorys_ti'].count() / data[data['categorys_ti'] == 'средний']['categorys_ti'].count()
part_ti_abovemean = data[(data['categorys_ti'] == 'выше среднего') & (data['debt'] == 1)]['categorys_ti'].count() / data[data['categorys_ti'] == 'выше среднего']['categorys_ti'].count()
part_ti_high = data[(data['categorys_ti'] == 'высокий') & (data['debt'] == 1)]['categorys_ti'].count() / data[data['categorys_ti'] == 'высокий']['categorys_ti'].count()
part_ti_veryhigh = data[(data['categorys_ti'] == 'очень высокий') & (data['debt'] == 1)]['categorys_ti'].count() / data[data['categorys_ti'] == 'очень высокий']['categorys_ti'].count()

print()
print('Доля людей с очень низким доходом, имевших задолженность по кредиту: {:.1%}'.format(part_ti_verylow))
print('Доля людей с низким доходом, имевших задолженность по кредиту: {:.1%}'.format(part_ti_low))
print('Доля людей со средним доходом, имевших задолженность по кредиту: {:.1%}'.format(part_ti_mean))
print('Доля людей с доходом выше среднего, имевших задолженность по кредиту: {:.1%}'.format(part_ti_abovemean))
print('Доля людей с высоким доходом, имевших задолженность по кредиту: {:.1%}'.format(part_ti_high))
print('Доля людей с очень высоким доходом, имевших задолженность по кредиту: {:.1%}'.format(part_ti_veryhigh))
print()


Доля людей с очень низким доходом, имевших задолженность по кредиту: 5.3%
Доля людей с низким доходом, имевших задолженность по кредиту: 6.9%
Доля людей со средним доходом, имевших задолженность по кредиту: 8.6%
Доля людей с доходом выше среднего, имевших задолженность по кредиту: 8.5%
Доля людей с высоким доходом, имевших задолженность по кредиту: 8.3%
Доля людей с очень высоким доходом, имевших задолженность по кредиту: 6.9%



### Вывод
Зависимость есть.
Больше всего доля должников среди людей со средним и выше среднего доходом, у бедных мало долгов, потому что они рациональнее используют средства и к кредиту подходят более ответственно брать / не брать, а богатые чаще всего финансово грамотные, поэтому
проблем с возвратом кредита в срок меньше.

- Как разные цели кредита влияют на его возврат в срок?

In [28]:
data['category'].value_counts()

недвижимость     8326
автомобиль       4306
образование      4013
свадьба          2324
строительство    1878
ремонт            607
Name: category, dtype: int64

In [29]:
part_ralty = data[(data['category'] == 'недвижимость') & (data['debt'] == 1)]['category'].count() / data[data['category'] == 'недвижимость']['category'].count()
part_car = data[(data['category'] == 'автомобиль') & (data['debt'] == 1)]['category'].count() / data[data['category'] == 'автомобиль']['category'].count()
part_education = data[(data['category'] == 'образование') & (data['debt'] == 1)]['category'].count() / data[data['category'] == 'образование']['category'].count()
part_wedding = data[(data['category'] == 'свадьба') & (data['debt'] == 1)]['category'].count() / data[data['category'] == 'свадьба']['category'].count()
part_build = data[(data['category'] == 'строительство') & (data['debt'] == 1)]['category'].count() / data[data['category'] == 'строительство']['category'].count()
part_repair = data[(data['category'] == 'ремонт') & (data['debt'] == 1)]['category'].count() / data[data['category'] == 'ремонт']['category'].count()

print()
print('Доля людей с целью кредита "недвижимость", имевших задолженность по кредиту: {:.1%}'.format(part_ralty))
print('Доля людей с целью кредита "автомобиль", имевших задолженность по кредиту: {:.1%}'.format(part_car))
print('Доля людей с целью кредита "образование", имевших задолженность по кредиту: {:.1%}'.format(part_education))
print('Доля людей с целью кредита "свадьба", имевших задолженность по кредиту: {:.1%}'.format(part_wedding))
print('Доля людей с целью кредита "строительство", имевших задолженность по кредиту: {:.1%}'.format(part_build))
print('Доля людей с целью кредита "ремонт", имевших задолженность по кредиту: {:.1%}'.format(part_repair))
print()


Доля людей с целью кредита "недвижимость", имевших задолженность по кредиту: 7.2%
Доля людей с целью кредита "автомобиль", имевших задолженность по кредиту: 9.4%
Доля людей с целью кредита "образование", имевших задолженность по кредиту: 9.2%
Доля людей с целью кредита "свадьба", имевших задолженность по кредиту: 8.0%
Доля людей с целью кредита "строительство", имевших задолженность по кредиту: 7.7%
Доля людей с целью кредита "ремонт", имевших задолженность по кредиту: 5.8%



### Вывод

Зависимость между целью кредита и возвратом в срок есть.

Попробую объяснить. Недвижимость и строительство - это обычно большие суммы и к ним люди подходят более ответственно, когда принимают решение брать или нет. Ремонт находится на нижней планке среди этих категорий по величине суммы, его просто легче закрыть. В середине (по величине суммы) находятся цели: атомобиль, образование, свадьба. Эти суммы не маленькие, но и не такие большие, чтобы ответственность была такая же, как у недвижимости, поэтому люди скорее всего не могут объективно оценить свои возможности и чаще рискуют. Также кредит на образование и автомобиль могут быть вынужденными мерами (неожиданно сломалась машина, а без нее никак; ребенок закончил школу и не поступил в вуз на бесплатное). Кредит на свадьбу можно взять на эмоциях + чаще такой кредит берут люди с низким доходом.